**Demo for `teneva.core.tensor`**

---

This module contains the basic operations and utilities for TT-tensors, including "add", "get", "mul", etc.

## Loading and importing modules

In [1]:
import numpy as np
import teneva
from time import perf_counter as tpc
np.random.seed(42)

## Function `accuracy`

Compute $\frac{|| Y_1 - Y_2 ||}{|| Y_2 ||}$ for tensors in the TT-format.

In [2]:
Y1 = teneva.rand([5]*10, 2)                 # 10-dim random TT-tensor with TT-rank 2
Y2 = teneva.add(Y1, teneva.mul(1.E-4, Y1))  # The TT-tensor Y1 + eps * Y1 (eps = 1.E-4)
eps = teneva.accuracy(Y1, Y2)               # The relative difference ("accuracy")
print(f'Accuracy     : {eps:-8.2e}')

Accuracy     : 1.00e-04


Note that this function works correctly even for very large dimension values due to the use of balancing in the scalar product:

In [3]:
for d in [10, 50, 100, 250, 1000, 10000]:
    Y1 = teneva.rand([10]*d, r=2)
    Y2 = teneva.add(Y1, Y1)

    eps = teneva.accuracy(Y1, Y2)

    print(f'd = {d:-5d} | eps = {eps:-8.1e} | expected value 0.5')

d =    10 | eps =  5.0e-01 | expected value 0.5
d =    50 | eps =  5.0e-01 | expected value 0.5
d =   100 | eps =  5.0e-01 | expected value 0.5
d =   250 | eps =  5.0e-01 | expected value 0.5
d =  1000 | eps =  5.0e-01 | expected value 0.5
d = 10000 | eps =  5.0e-01 | expected value 0.5


## Function `add`

Compute element wise sum $Y = Y_1 + Y_2$ for the given TT-tensors $Y_1$ and $Y_2$ presented as lists of TT-cores.

In [4]:
Y1 = teneva.rand([5]*10, 2) # 10-dim random TT-tensor with TT-rank 2
Y2 = teneva.rand([5]*10, 3) # 10-dim random TT-tensor with TT-rank 3

In [5]:
Y = teneva.add(Y1, Y2)      # Compute the sum of Y1 and Y2
teneva.show(Y)              # Print the resulting TT-tensor (note that it has TT-rank 2 + 3 = 5)

  5  5  5  5  5  5  5  5  5  5 
 / \/ \/ \/ \/ \/ \/ \/ \/ \/ \
 1  5  5  5  5  5  5  5  5  5  1 



In [6]:
Y1_full = teneva.full(Y1)                       # Compute tensors in the full format
Y2_full = teneva.full(Y2)                       # to check the result
Y_full = teneva.full(Y)

Z_full = Y1_full + Y2_full

e = np.linalg.norm(Y_full - Z_full)             # Compute error for TT-tensor vs full tensor 
e /= np.linalg.norm(Z_full)

print(f'Error     : {e:-8.2e}')                 # Rel. error for TT-tensor vs full tensor

Error     : 8.70e-17


This function also supports float argument:

In [7]:
Y1 = teneva.rand([5]*10, 2) # 10-dim random TT-tensor with TT-rank 2
Y2 = 42.                    # Just a number
Y = teneva.add(Y1, Y2)      # Compute the sum of Y1 and Y2
teneva.show(Y)              # Print the resulting TT-tensor (note that it has TT-rank 2 + 1 = 3)

  5  5  5  5  5  5  5  5  5  5 
 / \/ \/ \/ \/ \/ \/ \/ \/ \/ \
 1  3  3  3  3  3  3  3  3  3  1 



In [8]:
Y1 = 42.                    # Just a number
Y2 = teneva.rand([5]*10, 2) # 10-dim random TT-tensor with TT-rank 2
Y = teneva.add(Y1, Y2)      # Compute the sum of Y1 and Y2
teneva.show(Y)              # Print the resulting TT-tensor (note that it has TT-rank 2 + 1 = 3)

  5  5  5  5  5  5  5  5  5  5 
 / \/ \/ \/ \/ \/ \/ \/ \/ \/ \
 1  3  3  3  3  3  3  3  3  3  1 



In [9]:
Y1_full = 42.                       # Compute tensors in the full format
Y2_full = teneva.full(Y2)           # to check the result
Y_full = teneva.full(Y)

Z_full = Y1_full + Y2_full

e = np.linalg.norm(Y_full - Z_full) # Compute error for TT-tensor vs full tensor 
e /= np.linalg.norm(Z_full)

print(f'Error     : {e:-8.2e}')     # Rel. error for TT-tensor vs full tensor

Error     : 4.97e-16


If both arguments are numbers, then function returns the sum of numbers:

In [10]:
Y1 = 40.                    # Just a number
Y2 = 2                      # Just a number
Y = teneva.add(Y1, Y2)      # Compute the sum of Y1 and Y2
print(Y)                    # The result is a number

42.0


## Function `add_many`

Compute element wise sum $Y = Y_1 + Y_2 + \ldots + Y_m$ for the given TT-tensors given as lists of TT-cores. The result is truncated to the given accuracy `e` and/or maximum TT-rank `r`. Additionally, the intermediate result is truncated with a frequency `trunc_freq`.

In [11]:
Y_all = [teneva.rand([5]*10, 2) for _ in range(10)]     # 10 random TT-tensors with TT-rank 2
Y = teneva.add_many(Y_all, e=1.E-4, r=50, trunc_freq=2)
teneva.show(Y)

  5  5  5  5  5  5  5  5  5  5 
 / \/ \/ \/ \/ \/ \/ \/ \/ \/ \
 1  5 20 20 20 20 20 20 20  5  1 



This function also supports float arguments:

In [12]:
Y_all = [
    42.,
    teneva.rand([5]*10, 2),
    33.,
    teneva.rand([5]*10, 4)
]
Y = teneva.add_many(Y_all, e=1.E-4, r=50, trunc_freq=2)
teneva.show(Y)

  5  5  5  5  5  5  5  5  5  5 
 / \/ \/ \/ \/ \/ \/ \/ \/ \/ \
 1  5  7  7  7  7  7  7  7  5  1 



If all arguments are numbers, then function returns the sum of numbers:

In [13]:
Y_all = [10., 20., 2., 10.]
Y = teneva.add_many(Y_all, e=1.E-4, r=50, trunc_freq=2)
print(Y)

42.0


## Function `copy`

Return a copy of the given TT-tensor.

In [14]:
Y = teneva.rand([5]*10, 2) # 10-dim random TT-tensor with TT-rank 2
Z = teneva.copy(Y)         # The copy of Y
print(Y[2][1, 2, 0])
print(Z[2][1, 2, 0])

-0.1858103329563536
-0.1858103329563536


In [15]:
Z[2][1, 2, 0] = 42.

print(Y[2][1, 2, 0])
print(Z[2][1, 2, 0])

-0.1858103329563536
42.0


It also supports numbers for convenience:

In [16]:
teneva.copy(42.)

42.0

## Function `get`

Compute the element with multiindex $k$ from the TT-tensor $Y$. See also a function `tensor.getter` that performs the same operation, but with an acceleration.

In [17]:
n = [10] * 5              # Shape of the tensor      
Y0 = np.random.randn(*n)  # Create 5-dim random numpy tensor
Y1 = teneva.svd(Y0)       # Compute TT-tensor from Y0 by TT-SVD
teneva.show(Y1)           # Print the TT-tensor
k = [1, 2, 3, 4, 5]       # Select some tensor element
y1 = teneva.get(Y1, k)    # Compute the element of the TT-tensor
y0 = Y0[tuple(k)]         # Compute the same element of the original tensor
abs(np.max(y1-y0))        # Compare original tensor and reconstructed tensor

   10  10  10  10  10 
  / \ / \ / \ / \ / \ 
 1   10 100 100  10  1  



6.106226635438361e-16

This function is also support batch mode:

In [18]:
K = [
    [1, 2, 3, 4, 5],
    [0, 0, 0, 0, 0],
    [5, 4, 3, 2, 1],
]

y1 = teneva.get(Y1, k)
y0 = [Y0[tuple(k)] for k in K]
abs(np.max(y1-y0))

1.9302352470832727

## Function `get_many`

Compute the elements of the TT-tensor on many indices (the function "get" with 2-dimensional argument may be used instead).

In [19]:
n = [10] * 5                    # Shape of the tensor      
Y0 = np.random.randn(*n)        # Create 5-dim random numpy tensor
Y1 = teneva.svd(Y0)             # Compute TT-tensor from Y0 by TT-SVD
teneva.show(Y1)                 # Print the TT-tensor
K = [                           # Select some tensor elements
    [1, 2, 3, 4, 5],
    [0, 0, 0, 0, 0],
    [5, 4, 3, 2, 1],
]     
y1 = teneva.get_many(Y1, K)     # Compute the element of the TT-tensor
y0 = [Y0[tuple(k)] for k in K]  # Compute the same element of the original tensor
abs(np.max(y1-y0))              # Compare original tensor and reconstructed tensor

   10  10  10  10  10 
  / \ / \ / \ / \ / \ 
 1   10 100 100  10  1  



1.2212453270876722e-14

## Function `getter`

Build fast (accelerated by numba) function that computes the element with multiindex $k$ for the TT-tensor $Y$. See also a function `tensor.get` for more details. Note that this function is not support the batch mode.

In [20]:
n = [10] * 5              # Shape of the tensor      
Y0 = np.random.randn(*n)  # Create 5-dim random numpy tensor
Y1 = teneva.svd(Y0)       # Compute TT-tensor from Y0 by TT-SVD
get = teneva.getter(Y1)   # Build (compile) function to compute the element of the TT-tensor
k = (1, 2, 3, 4, 5)       # Select some tensor element
y1 = get(k)               # Compute the element of the TT-tensor
y0 = Y0[k]                # Compute the same element of the original tensor
abs(np.max(y1-y0))        # Compare original tensor and reconstructed tensor

1.9897278269453977e-15

## Function `mul`

Compute element wise product $Y = Y_1 + Y_2$ for the given TT-tensors $Y_1$ and $Y_2$ presented as lists of TT-cores.

In [21]:
Y1 = teneva.rand([5]*10, 2) # 10-dim random TT-tensor with TT-rank 2
Y2 = teneva.rand([5]*10, 3) # 10-dim random TT-tensor with TT-rank 3

In [22]:
Y = teneva.mul(Y1, Y2)      # Compute the product of Y1 and Y2
teneva.show(Y)              # Print the resulting TT-tensor (note that it has TT-rank 2 x 3 = 6)

  5  5  5  5  5  5  5  5  5  5 
 / \/ \/ \/ \/ \/ \/ \/ \/ \/ \
 1  6  6  6  6  6  6  6  6  6  1 



In [23]:
Y1_full = teneva.full(Y1)                       # Compute tensors in the full format
Y2_full = teneva.full(Y2)                       # to check the result
Y_full = teneva.full(Y)

Z_full = Y1_full * Y2_full

e = np.linalg.norm(Y_full - Z_full)             # Compute error for TT-tensor vs full tensor 
e /= np.linalg.norm(Z_full)                     #

print(f'Error     : {e:-8.2e}')                 # Rel. error for TT-tensor vs full tensor

Error     : 4.03e-16


This function also supports float argument:

In [24]:
Y1 = teneva.rand([5]*10, 2) # 10-dim random TT-tensor with TT-rank 2
Y2 = 42.                    # Just a number
Y = teneva.mul(Y1, Y2)      # Compute the product of Y1 and Y2
teneva.show(Y)              # Print the resulting TT-tensor (note that it has TT-rank 2 x 1 = 2)

  5  5  5  5  5  5  5  5  5  5 
 / \/ \/ \/ \/ \/ \/ \/ \/ \/ \
 1  2  2  2  2  2  2  2  2  2  1 



In [25]:
Y1 = 42.                    # Just a number
Y2 = teneva.rand([5]*10, 2) # 10-dim random TT-tensor with TT-rank 2
Y = teneva.mul(Y1, Y2)      # Compute the product of Y1 and Y2
teneva.show(Y)              # Print the resulting TT-tensor (note that it has TT-rank 2 x 1 = 2)

  5  5  5  5  5  5  5  5  5  5 
 / \/ \/ \/ \/ \/ \/ \/ \/ \/ \
 1  2  2  2  2  2  2  2  2  2  1 



In [26]:
Y1 = 21.                    # Just a number
Y2 = 2                      # Just a number
Y = teneva.mul(Y1, Y2)      # Compute the product of Y1 and Y2
print(Y)                    # The result is a number

42.0


## Function `mul_scalar`

Compute scalar product for Y1 and Y2 in the TT-format.

In [27]:
Y1 = teneva.rand([5]*10, 2)           # 10-dim random TT-tensor with TT-rank 2
Y2 = teneva.rand([5]*10, 3)           # 10-dim random TT-tensor with TT-rank 3

In [28]:
v = teneva.mul_scalar(Y1, Y2)         # Compute the product of Y1 and Y2
print(v)                              # Print the resulting value

-1335426.1415004898


In [29]:
Y1_full = teneva.full(Y1)             # Compute tensors in the full format
Y2_full = teneva.full(Y2)             # to check the result

v_full = np.sum(Y1_full * Y2_full)
print(v_full)                         # Print the resulting value from full tensor

e = abs((v - v_full)/v_full)          # Compute error for TT-tensor vs full tensor 
print(f'Error     : {e:-8.2e}')       # Rel. error

-1335426.1415004958
Error     : 4.53e-15


We can also set a flag "use_stab", in which case a value that is 2^p times smaller than the real value will be returned:

In [30]:
v, p = teneva.mul_scalar(Y1, Y2, use_stab=True)
print(v)
print(p)
print(v*2**p)

-1.2735616126065157
20
-1335426.1415004898


## Function `rand`

Construct random TT-tensor.

In [31]:
n = [12, 13, 14, 15, 16]    # Shape of the tensor
r = [1, 2, 3, 4, 5, 1]      # TT-ranks for TT-tensor
Y = teneva.rand(n, r)       # Build random TT-tensor
teneva.show(Y)              # Print the resulting TT-tensor

 12 13 14 15 16 
 / \/ \/ \/ \/ \
 1  2  3  4  5  1 



If all inner TT-ranks are equal, we may pass it as a number:

In [32]:
n = [12, 13, 14, 15, 16]    # Shape of the tensor
r = 5                       # TT-ranks for TT-tensor
Y = teneva.rand(n, r)       # Build random TT-tensor
teneva.show(Y)              # Print the resulting TT-tensor

 12 13 14 15 16 
 / \/ \/ \/ \/ \
 1  5  5  5  5  1 



## Function `show`

Displays mode sizes and TT-ranks of the given TT-tensor Y in a compact and clear form.

In [33]:
Y = teneva.rand([10, 12, 8, 8, 30], 2) # 5-dim random TT-tensor with TT-rank 2
teneva.show(Y)                         # Print the resulting TT-tensor

 10 12  8  8 30 
 / \/ \/ \/ \/ \
 1  2  2  2  2  1 



## Function `sub`

Compute element wise difference $Y = Y_1 - Y_2$ for the given TT-tensors $Y_1$ and $Y_2$ presented as lists of TT-cores.

In [34]:
Y1 = teneva.rand([5]*10, 2) # 10-dim random TT-tensor with TT-rank 2
Y2 = teneva.rand([5]*10, 3) # 10-dim random TT-tensor with TT-rank 3

In [35]:
Y = teneva.sub(Y1, Y2)      # Compute the difference between Y1 and Y2
teneva.show(Y)              # Print the resulting TT-tensor (note that it has TT-rank 2 + 3 = 5)

  5  5  5  5  5  5  5  5  5  5 
 / \/ \/ \/ \/ \/ \/ \/ \/ \/ \
 1  5  5  5  5  5  5  5  5  5  1 



In [36]:
Y1_full = teneva.full(Y1)                       # Compute tensors in the full format
Y2_full = teneva.full(Y2)                       # to check the result
Y_full = teneva.full(Y)

Z_full = Y1_full - Y2_full

e = np.linalg.norm(Y_full - Z_full)             # Compute error for TT-tensor vs full tensor 
e /= np.linalg.norm(Z_full)                     

print(f'Error     : {e:-8.2e}')                 # Rel. error for TT-tensor vs full tensor

Error     : 8.30e-17


This function also supports float argument:

In [37]:
Y1 = teneva.rand([5]*10, 2) # 10-dim random TT-tensor with TT-rank 2
Y2 = 42.                    # Just a number
Y = teneva.sub(Y1, Y2)      # Compute the difference between Y1 and Y2
teneva.show(Y)              # Print the resulting TT-tensor (note that it has TT-rank 2 + 1 = 3)

  5  5  5  5  5  5  5  5  5  5 
 / \/ \/ \/ \/ \/ \/ \/ \/ \/ \
 1  3  3  3  3  3  3  3  3  3  1 



In [38]:
Y1 = 42.                    # Just a number
Y2 = teneva.rand([5]*10, 2) # 10-dim random TT-tensor with TT-rank 2
Y = teneva.sub(Y1, Y2)      # Compute the difference between Y1 and Y2
teneva.show(Y)              # Print the resulting TT-tensor (note that it has TT-rank 2 + 1 = 3)

  5  5  5  5  5  5  5  5  5  5 
 / \/ \/ \/ \/ \/ \/ \/ \/ \/ \
 1  3  3  3  3  3  3  3  3  3  1 



In [39]:
Y1 = 44.                    # Just a number
Y2 = 2                      # Just a number
Y = teneva.sub(Y1, Y2)      # Compute the difference between Y1 and Y2
print(Y)                    # The result is a number

42.0


## Function `sum`

Compute sum of all tensor elements.

In [40]:
Y = teneva.rand([10, 12, 8, 8, 30], 2) # 5-dim random TT-tensor with TT-rank 2
teneva.sum(Y)                          # Sum of the TT-tensor elements

-104.87021259002125

In [41]:
Z = teneva.full(Y)                     # Compute tensors in the full format to check the result
np.sum(Z)

-104.87021259002152

---